In [1]:
# https://www.kaggle.com/timesler/fast-mtcnn-detector-45-fps-at-full-resolution

In [4]:
! pip install imutils

  Created wheel for imutils: filename=imutils-0.5.3-cp37-none-any.whl size=25850 sha256=7fc14586dfd314caa2d5d3fbc90aa95814cc4b1f3696bebff0dfcaecbc654607
  Stored in directory: /home/jupyter/.cache/pip/wheels/16/84/1f/bf88641293cda2c8be81a5c4b8ca973dd9125a6dc3767417fd
Successfully built imutils


In [2]:
%%capture
# Install facenet-pytorch (with internet use "pip install facenet-pytorch")
!pip install facenet-pytorch

In [6]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch
from imutils.video import FileVideoStream
import cv2
import time
import glob
from tqdm import tqdm_notebook

device = 'cuda' if torch.cuda.is_available() else 'cpu'

filenames = glob.glob('data/videos/*.mp4')

In [2]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Fac8es will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frames):
        """Detect faces in frames using strided MTCNN."""
        if self.resize != 1:
            frames = [f.resize([int(d * self.resize) for d in f.size]) for f in frames]
                      
        boxes, probs = self.mtcnn.detect(frames[::self.stride])

        faces = []
        for i, frame in enumerate(frames):
            box_ind = int(i / self.stride)
            if boxes[box_ind] is None:
                continue
            for box in boxes[box_ind]:
                faces.append(frame.crop(box))
        
        return faces

In [13]:
import matplotlib.pyplot as plt
import shutil
import os
%matplotlib inline

FACES_PATH = '/home/jupyter/faces/'
try:
    shutil.rmtree(FACES_PATH)
except:
    pass


try: 
    os.mkdir(FACES_PATH)
except:
    pass

def run_detection(fast_mtcnn, filenames):
    
    frames = []
    frames_processed = 0
    faces_detected = 0
    batch_size = 60
    framePass = 5
    start = time.time()

    for filename in tqdm_notebook(filenames):
        try: 
            name = filename.split('/')[-1]
            os.mkdir(FACES_PATH + name)
        except:
            pass
        v_cap = FileVideoStream(filename).start()
        v_len = int(v_cap.stream.get(cv2.CAP_PROP_FRAME_COUNT))

        for j in range(v_len):
            frame = v_cap.read()
            if j % framePass != 0 and j > 0:
                continue
            try:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
            except:
                break
            
            frames.append(frame)

            if len(frames) >= batch_size or j == v_len - 1:
                try:
                    faces = fast_mtcnn(frames)
                except:
                    frames = []
                    break
                
                for index, face in enumerate(faces):
                    name = filename.split('/')[-1]
                    number = index * framePass
                    face.save(FACES_PATH + name+'/'+str(number)+'.png')
                    
                frames_processed += len(frames)
                faces_detected += len(faces)
                frames = []
                
#                 print('FPS, tot')
#                 print(
#                     frames_processed / (time.time() - start),
#                     faces_detected
#                 )

        v_cap.stop()

In [4]:
# fast_mtcnn = FastMTCNN(
#     stride=4,
#     resize=1,
#     margin=14,
#     factor=0.6,
#     keep_all=True,
#     device=device
# )
# run_detection(fast_mtcnn, filenames)

In [14]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=0.5,
    margin=14,
    factor=0.6,
    keep_all=True,
    device=device
)
run_detection(fast_mtcnn, filenames)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


KeyboardInterrupt: 